<a href="https://colab.research.google.com/github/hannape/CNN-pierwsze-podejscie/blob/master/cnn_scorer_not_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title
import keras
from sklearn.metrics import roc_auc_score

class Histories(keras.callbacks.Callback):
	def on_train_begin(self, logs={}):
		self.aucs = []
		self.losses = []

	def on_train_end(self, logs={}):
		return

	def on_epoch_begin(self, epoch, logs={}):
		return

	def on_epoch_end(self, epoch, logs={}):
		self.losses.append(logs.get('loss'))
		y_pred = self.model.predict(self.validation_data[0])
		self.aucs.append(roc_auc_score(self.validation_data[1], y_pred))
		a = (roc_auc_score(self.validation_data[1], y_pred))
		print(" AUC_on_val: %f " % a)
		return

	def on_batch_begin(self, batch, logs={}):
		return

	def on_batch_end(self, batch, logs={}):    return
  
from google.colab import drive
drive.mount('/content/drive')


In [231]:
from __future__ import print_function

import keras
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.utils import class_weight
from sklearn.metrics import  make_scorer
from sklearn.metrics import log_loss
K.set_image_dim_ordering('th')
import numpy as np
from sklearn import preprocessing
import glob
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# num_classes = 2
X_train = numpy.load('drive/My Drive/X_train.npy')
y_train = numpy.load('drive/My Drive/y_train.npy')
X_test = numpy.load('drive/My Drive/X_test.npy')
y_test = numpy.load('drive/My Drive/y_test.npy')

X_train = X_train.reshape(X_train.shape[0], 1, 60, 87).astype('float32')
#X_val = X_val.reshape(X_val.shape[0], 1, 60, 87).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 60, 87).astype('float32')

input_shape = (1, 60, 87)
# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(30581, 1, 60, 87) (30581,)
(10793, 1, 60, 87) (10793,)


In [0]:
#@title
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size,input_shape=input_shape, activation='relu'))
    #model.add(Activation('relu'))
    #model.add(Conv2D(filters, kernel_size))
    #model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    #for layer_size in dense_layer_sizes:
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model



In [262]:
c_w = { 0:0.02, 1:0.98}
index = ['r%d' % x for x in range(len(y_train))]
a = [c_w[class_label] for class_label in y_train]
sample_weight_frame = pd.DataFrame(a, index = index)
score_params = {"sample_weight": sample_weight_frame}
print(sample_weight_frame[0:5])

       0
r0  0.98
r1  0.02
r2  0.02
r3  0.02
r4  0.02


In [0]:
#class_weights = class_weight.compute_class_weight('balanced',numpy.unique(y_train),y_train)
import functools
def my_score(y_true, y_pred, sample_weight): 
  return log_loss(y_true.values, y_pred, sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1), normalize=True)
    
def my_score2(sample_weight):  # <--------------------------------wersja 2, implementowana w ten sposób by potem przy compile(loss=...) nie trzeba było podawać 3 argumentów (bo nie wiem jak nawet gdybym chciała)
  def loss_2_args(y_true, y_pred):  
      return log_loss(y_true.values, y_pred, sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1), normalize=True)   

  return loss_2_args

my_scorer = make_scorer(my_score,
                          greater_is_better=False, 
                          needs_threshold=False,
                          **score_params)

my_scorer2 = make_scorer(my_score2,
                          greater_is_better=False, 
                          needs_threshold=False,
                          **score_params)

my_loss = functools.partial(my_score, sample_weight=sample_weight_frame)  # za: https://github.com/keras-team/keras/issues/2115

In [0]:
#@title
import tensorflow

from tensorflow.python.ops import clip_ops
from tensorflow.python.framework import dtypes as dtypes_module
from tensorflow.python.util.tf_export import tf_export
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn
@tf_export('keras.backend.dtype')
def dtype(x):
  return x.dtype.base_dtype.name

_EPSILON = 1e-7
def epsilon():
  return _EPSILON

def _to_tensor(x, dtype):
  return ops.convert_to_tensor(x, dtype=dtype)

def binary_crossentropy(target, output, from_logits=False):
  if not from_logits:
    # transform back to logits
   
    #epsilon_ = tensorflow.convert_to_tensor(epsilon(), np.float32)
    #output = clip_ops.clip_by_value(output, epsilon_, 1 - epsilon_)
    output = math_ops.log(output / (1 - output))
  return nn.sigmoid_cross_entropy_with_logits(labels=target, logits=output)

In [0]:
#del model
def make_model_modified(dense_layer_sizes,filters, kernel_size,pool_size):

    #dense_layer_sizes= 128
    #filters = 20
    #kernel_size = [3,3]
    #pool_size = [2,2]
    score_params = {"sample_weight": sample_weight_frame}
    
    model = Sequential()
    model.add(Conv2D(filters, kernel_size,input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Conv2D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Conv2D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
   
    model.add(Flatten())
    model.add(Dense(dense_layer_sizes, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', # my_score2(sample_weight = sample_weight_frame), <------------------- nie umiem zrobić by działał tutaj. Która funkcja? my_score czy my_scorer ?
                  # co tu powinno wejść? o samo co do scoring w GridSearchCv? chyba nie: https://stackoverflow.com/questions/40572743/scikit-learn-grid-search-own-scoring-object-syntax
                  # my_score2(sample_weight_frame) ---->  https://stackoverflow.com/questions/46858016/keras-custom-loss-function-to-pass-arguments-other-than-y-true-and-y-pred
                  # my_loss ---------------------------->też nie chodzi a chyba powinien, coś z formatem nie tak.  ERROR: 'Tensor' object has no attribute 'values'
                  # inne źródło: https://github.com/keras-team/keras/issues/2115
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [0]:
#@title
## spr czemu się wykrzacza
model = make_model_modified() #(128,20,3,2)
hist = model.fit(X_train, y_train, epochs=15, batch_size=64, callbacks=[histories]) # validation_data=(X_val, y_val), class_weight = class_weights


In [0]:
#@title
def larger_model():
  model = Sequential()
  model.add(Conv2D(20, (3,3), input_shape=(1, 60, 87), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(20, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(20, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  #model.compile(loss=roc_auc_score_loss, optimizer='adam', metrics=['accuracy','mae'])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

  

In [0]:
# build the model
my_classifier = KerasClassifier(make_model_modified)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': [128],
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [8],
                                     'filters': [20],
                                     'kernel_size': [(3,3)],
                                     'pool_size': [(2,2)],
                                     # 'class_weight': [{ 0:0.02, 1:0.98}],  <------------------------------- może w ten sposób? Ale t też tylko w grid searchu
                                     'batch_size': [32, 64]}, 
                         scoring=my_scorer, n_jobs=1, refit=True, cv=2)  ## SCORING ZMIENIONY na my_scorer (inne, np: 'roc_auc')


In [0]:
histories = Histories()

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',numpy.unique(y_train),y_train)
print(class_weights)

y_frame = pd.DataFrame(y_train, index=index)
grid_result = validator.fit(X_train, y_frame) #, class_weight = class_weights)

[ 0.50930984 27.35330948]
Epoch 1/8
15290/15290 [==============================] - 11s 721us/step - loss: 0.1107 - acc: 0.9864
Epoch 2/8
15290/15290 [==============================] - 8s 513us/step - loss: 0.0690 - acc: 0.9885
Epoch 3/8
15290/15290 [==============================] - 8s 514us/step - loss: 0.0692 - acc: 0.9885
Epoch 4/8
15290/15290 [==============================] - 8s 507us/step - loss: 0.0679 - acc: 0.9886
Epoch 5/8
15290/15290 [==============================] - 8s 511us/step - loss: 0.0684 - acc: 0.9884
Epoch 6/8
15290/15290 [==============================] - 8s 502us/step - loss: 0.0670 - acc: 0.9885
Epoch 7/8
15290/15290 [==============================] - 8s 508us/step - loss: 0.0678 - acc: 0.9884
Epoch 8/8
15290/15290 [==============================] - 8s 511us/step - loss: 0.0676 - acc: 0.9886
Epoch 1/8
15291/15291 [==============================] - 11s 736us/step - loss: 0.1345 - acc: 0.9744
Epoch 2/8
15291/15291 [==============================] - 8s 517us/step -